In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("info.csv")

In [3]:
df.head()

,Unnamed: 0,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,0,2,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,1,2,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,2,2,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,3,2,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,4,2,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground


In [4]:
df.shape

(63888, 9)

In [5]:
df[df['city'].isnull()]['venue'].value_counts()

venue
Dubai International Cricket Stadium        2969
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sharjah Cricket Stadium                     249
Sylhet International Cricket Stadium        128
Carrara Oval                                 64
Name: count, dtype: int64

In [6]:
x = np.where(df['city'].isnull(), df['venue'].str.split().apply(lambda x:x[0]), df['city'])

In [7]:
df['city'] = x

In [8]:
df.isnull().sum()

Unnamed: 0          0
match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [9]:
df.drop(columns=['Unnamed: 0','venue'],inplace=True)

In [10]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne
...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo


In [11]:
total_df = df.groupby('match_id').sum()['runs'].reset_index()
df = df.merge(total_df,on='match_id')

In [12]:
df

,match_id,batting_team,bowling_team,ball,runs_x,player_dismissed,city,runs_y
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,168
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,168
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,168
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,168
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,168
...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,19.3,1,0,Colombo,128
63884,964,Sri Lanka,Australia,19.4,0,0,Colombo,128
63885,964,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,128
63886,964,Sri Lanka,Australia,19.6,2,0,Colombo,128


In [13]:
df['runs_x'] = pd.to_numeric(df['runs_x'], errors='coerce')
df['current_score'] = df.groupby('match_id')['runs_x'].cumsum()


In [14]:
df['over'] = df['ball'].apply(lambda x:str(x).split(".")[0])
df['ball_no'] = df['ball'].apply(lambda x:str(x).split(".")[1])

In [15]:
df['balls_bowled'] = (df['over'].astype('int')*6) + df['ball_no'].astype('int')

In [16]:
df['crr'] = round((df['current_score']*6)/df['balls_bowled'],2)

In [17]:
# df['player_dismissed'] = df['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
# df['player_dismissed'] = df['player_dismissed'].astype('int')
# df['player_dismissed'] = df.groupby('match_id').cumsum()['player_dismissed']
# df['wickets_left'] = 10 - df['player_dismissed']

df['player_dismissed'] = df['player_dismissed'].apply(lambda x: 0 if x == '0' else 1)
df['player_dismissed'] = df.groupby('match_id')['player_dismissed'].cumsum()
df['wickets_left'] = 10 - df['player_dismissed']


In [18]:
final_df = df[['match_id','batting_team','bowling_team','runs_x','current_score','balls_bowled','wickets_left','crr','city','runs_y']]

In [19]:
final_df

,match_id,batting_team,bowling_team,runs_x,current_score,balls_bowled,wickets_left,crr,city,runs_y
0,2,Australia,Sri Lanka,0,0,1,10,0.00,Melbourne,168
1,2,Australia,Sri Lanka,0,0,2,10,0.00,Melbourne,168
2,2,Australia,Sri Lanka,1,1,3,10,2.00,Melbourne,168
3,2,Australia,Sri Lanka,2,3,4,10,4.50,Melbourne,168
4,2,Australia,Sri Lanka,0,3,5,10,3.60,Melbourne,168
...,...,...,...,...,...,...,...,...,...,...
63883,964,Sri Lanka,Australia,1,125,117,2,6.41,Colombo,128
63884,964,Sri Lanka,Australia,0,125,118,2,6.36,Colombo,128
63885,964,Sri Lanka,Australia,0,125,119,1,6.30,Colombo,128
63886,964,Sri Lanka,Australia,2,127,120,1,6.35,Colombo,128


In [20]:
final_df = final_df.sample(final_df.shape[0])

In [21]:
final_df['balls_left'] = 120 - final_df['balls_bowled']
final_df['balls_left'] = final_df['balls_left'].apply(lambda x:0 if x<0 else x)

In [22]:
final_df['crr'] = round((final_df['current_score']*6)/final_df['balls_bowled'],2)

In [23]:
final_df

,match_id,batting_team,bowling_team,runs_x,current_score,balls_bowled,wickets_left,crr,city,runs_y,balls_left
4688,64,India,West Indies,1,164,107,5,9.20,Jamaica,190,13
54863,820,South Africa,West Indies,2,160,109,8,8.81,Durban,195,11
4769,65,Sri Lanka,India,1,98,66,7,8.91,Colombo,170,54
31174,540,India,New Zealand,1,99,77,4,7.71,Christchurch,162,43
15618,232,India,Bangladesh,6,60,54,8,6.67,Delhi,148,66
...,...,...,...,...,...,...,...,...,...,...,...
56160,832,South Africa,Bangladesh,1,68,45,10,9.07,Mirpur,169,75
62690,942,India,Sri Lanka,1,161,102,7,9.47,Ranchi,196,18
44225,675,Pakistan,Bangladesh,0,6,6,10,6.00,Mirpur,135,114
13440,165,England,West Indies,1,30,27,7,6.67,Basseterre,182,93


In [24]:
final_df.drop(columns=['balls_bowled'],inplace=True)

In [25]:
groups = final_df.groupby('match_id')

In [26]:
# Convert 'runs_x' column to numeric, replacing non-numeric values with NaN
df['runs_x'] = pd.to_numeric(df['runs_x'], errors='coerce')

# Use rolling window to calculate the sum of the last 30 values in 'runs_x'
match_ids = final_df['match_id'].unique()
last_five = []
for match_id in match_ids:
    group = groups.get_group(match_id)
    last_five.extend(group['runs_x'].rolling(window=30, min_periods=1).sum().tolist())


In [27]:
final_df['last_five'] = last_five

In [28]:
final_df

,match_id,batting_team,bowling_team,runs_x,current_score,wickets_left,crr,city,runs_y,balls_left,last_five
4688,64,India,West Indies,1,164,5,9.20,Jamaica,190,13,1.0
54863,820,South Africa,West Indies,2,160,8,8.81,Durban,195,11,1.0
4769,65,Sri Lanka,India,1,98,7,8.91,Colombo,170,54,5.0
31174,540,India,New Zealand,1,99,4,7.71,Christchurch,162,43,6.0
15618,232,India,Bangladesh,6,60,8,6.67,Delhi,148,66,7.0
...,...,...,...,...,...,...,...,...,...,...,...
56160,832,South Africa,Bangladesh,1,68,10,9.07,Mirpur,169,75,47.0
62690,942,India,Sri Lanka,1,161,7,9.47,Ranchi,196,18,47.0
44225,675,Pakistan,Bangladesh,0,6,10,6.00,Mirpur,135,114,48.0
13440,165,England,West Indies,1,30,7,6.67,Basseterre,182,93,2.0


In [29]:
final_df.dropna(inplace=True)

In [30]:
final_df

,match_id,batting_team,bowling_team,runs_x,current_score,wickets_left,crr,city,runs_y,balls_left,last_five
4688,64,India,West Indies,1,164,5,9.20,Jamaica,190,13,1.0
54863,820,South Africa,West Indies,2,160,8,8.81,Durban,195,11,1.0
4769,65,Sri Lanka,India,1,98,7,8.91,Colombo,170,54,5.0
31174,540,India,New Zealand,1,99,4,7.71,Christchurch,162,43,6.0
15618,232,India,Bangladesh,6,60,8,6.67,Delhi,148,66,7.0
...,...,...,...,...,...,...,...,...,...,...,...
56160,832,South Africa,Bangladesh,1,68,10,9.07,Mirpur,169,75,47.0
62690,942,India,Sri Lanka,1,161,7,9.47,Ranchi,196,18,47.0
44225,675,Pakistan,Bangladesh,0,6,10,6.00,Mirpur,135,114,48.0
13440,165,England,West Indies,1,30,7,6.67,Basseterre,182,93,2.0


In [31]:
eligible_cities = final_df['city'].value_counts()[final_df['city'].value_counts() > 600].index.tolist()
final_df = final_df[final_df['city'].isin(eligible_cities)]

In [32]:
X = final_df.drop(columns=['match_id','runs_x','runs_y'])
y = final_df['runs_y']

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [34]:
X_test

,batting_team,bowling_team,current_score,wickets_left,crr,city,balls_left,last_five
59540,South Africa,Sri Lanka,34,10,8.87,Cape Town,97,19.0
58504,New Zealand,Sri Lanka,66,10,9.66,Mount Maunganui,79,42.0
12672,New Zealand,Pakistan,23,10,9.20,Dubai,105,36.0
36535,Sri Lanka,India,19,9,8.14,Chandigarh,106,30.0
23392,West Indies,South Africa,145,10,10.88,Johannesburg,40,45.0
...,...,...,...,...,...,...,...,...
38880,India,West Indies,17,9,4.86,Trinidad,99,33.0
44859,West Indies,New Zealand,22,9,12.00,Lauderhill,109,31.0
52821,India,Australia,91,6,6.66,Mirpur,38,40.0
11152,Bangladesh,West Indies,154,6,8.40,Lauderhill,10,43.0


In [35]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error

In [36]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')

In [37]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state=1))
])

In [38]:
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.9599340993626961
2.7239368711765564


In [39]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))